In [1]:
import keras
import tensorflow as tf
print('TensorFlow version:', tf.__version__) 
print('Keras version:', keras.__version__)
import sys
import os
from os.path import join
import json
import random
import itertools
import re
import datetime
#import cairocffi as cairo
import editdistance
import numpy as np
from scipy import ndimage
import pylab
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model, load_model
from keras.layers.recurrent import GRU
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import keras.callbacks
import cv2


Using TensorFlow backend.


TensorFlow version: 1.8.0
Keras version: 2.2.2


In [2]:
sess = tf.Session()
K.set_session(sess)
from keras.models import model_from_json

In [3]:
sys.path.append(os.path.dirname(os.path.realpath('Textgen')) + "/python")
import Textgen as t

Max plate length in "anpr_ocr__train": 8
Max plate length in "anpr_ocr__train": 8
33 33 33
Letters: 0 1 2 3 4 5 6 7 8 9 A B C D E F G H I J K L M N O P R S T U V Y Z
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z']
33
letters_val: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z']


# LOAD MODEL 

In [4]:
# load json and create model
fmodel='lr0.02_mom0.9'

json_file = open(fmodel + '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(fmodel + '.h5')
print("Loaded model from disk") 

Loaded model from disk


In [5]:
sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)
# Compile model
loaded_model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=sgd)

In [6]:
import matplotlib.pyplot as plt
# Get training and test loss histories
training_loss = loaded_model['loss']
test_loss = loaded_model.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

TypeError: 'Model' object is not subscriptable

# Test on validation images

In [7]:
tiger_test = t.TextImageGenerator('../data/anpr_ocr__test', 'test', 128, 64, 8, 4) # batch size 
tiger_test.build_data()

net_inp = loaded_model.get_layer(name='the_input').input
net_out = loaded_model.get_layer(name='softmax').output

In [8]:
net_inp,net_out

(<tf.Tensor 'the_input:0' shape=(?, 128, 64, 1) dtype=float32>,
 <tf.Tensor 'softmax/truediv:0' shape=(?, 32, 34) dtype=float32>)

In [9]:
tiger_test.batch_size,tiger_test.img_h,tiger_test.img_w

(8, 64, 128)

In [10]:
for inp_value, _ in tiger_test.next_batch(K):   
    nbr_example = inp_value['the_input'].shape[0]    
    print('the_input:',np.shape(inp_value['the_input']))
    print('the_labels:',np.shape(inp_value['the_labels']))
    print('input_length:',np.shape(inp_value['input_length']))
    print('label_length:',np.shape(inp_value['label_length']))
    for i  in range(nbr_example):
        print("labels:", inp_value['the_labels'][i])
    break

the_input: (8, 128, 64, 1)
the_labels: (8, 8)
input_length: (8, 1)
label_length: (8, 1)
labels: [ 0.  1. 11. 17.  3.  0.  7. 34.]
labels: [ 0.  1. 11. 17.  3.  0.  7. 34.]
labels: [ 0.  1. 11. 17.  3.  0.  7. 34.]
labels: [ 0.  1. 13. 14. 20.  1.  1. 34.]
labels: [ 0.  1. 13. 15. 10.  0.  3. 34.]
labels: [ 0.  1. 13. 18. 20.  9.  7. 34.]
labels: [ 0.  1. 13. 19. 22.  5.  2. 34.]
labels: [ 0.  3. 15. 12.  1.  8.  4. 34.]


#  Full test set

In [18]:
tiger_test = t.TextImageGenerator('../data/anpr_ocr__test', 'test', 128, 64, 461, 4) # batch size 
tiger_test.build_data()

In [19]:
for inp_value, _ in tiger_test.next_batch(K):
    matched=0
    non_matched=0
    testset_cnt = inp_value['the_input'].shape[0]
    X_data = inp_value['the_input']
    
    net_out_value = sess.run(net_out, feed_dict={net_inp:X_data})
    pred_texts = t.decode_batch(net_out_value)
    
    labels = inp_value['the_labels']
    texts = []     
    
    for label in labels:
        text = t.labels_to_text(label)        
        texts.append(text)       
    
    for cnt in range(testset_cnt):        
        
        if pred_texts[cnt] == texts[cnt]:
            matched += 1            
        else:
            non_matched +=  1
            print('wrong Predicted: %s True: %s i: %s len_pred: %s len_true: %s' % (pred_texts[cnt], texts[cnt],str(cnt),
                                                                              len(pred_texts[cnt]),len(texts[cnt])))
            
    print("matched=",matched, ",total=",testset_cnt , "TP: %", round(matched/testset_cnt, 2) * 100)
    break
    

wrong Predicted: 06BB17 True: 06BHB17 i: 13 len_pred: 6 len_true: 7
wrong Predicted: 06BCD70 True: 06BID70 i: 16 len_pred: 7 len_true: 7
wrong Predicted: 06DK5309 True: 06DK5303 i: 24 len_pred: 8 len_true: 8
wrong Predicted: 06DK507 True: 06DK5907 i: 29 len_pred: 7 len_true: 8
wrong Predicted: 34Y1288 True: 13TN1268 i: 40 len_pred: 7 len_true: 8
wrong Predicted: 41YG0941 True: 41YC004 i: 61 len_pred: 8 len_true: 7
wrong Predicted: 43HP98 True: 43HP983 i: 74 len_pred: 6 len_true: 7
wrong Predicted: 52K35088 True: 52K3508 i: 76 len_pred: 8 len_true: 7
wrong Predicted: 81EN209 True: 81EH209 i: 82 len_pred: 7 len_true: 7
wrong Predicted: 01DFA25 True: 01DFA251 i: 87 len_pred: 7 len_true: 8
wrong Predicted: 06BAF319 True: 06BJF37 i: 117 len_pred: 8 len_true: 7
wrong Predicted: 6BP7019 True: 06BR7019 i: 118 len_pred: 7 len_true: 8
wrong Predicted: 08FJ7246 True: 06FJ7246 i: 137 len_pred: 8 len_true: 8
wrong Predicted: 06HC6P True: 06HC609 i: 142 len_pred: 6 len_true: 7
wrong Predicted: 06JDF

In [11]:
# evaluate the model
"""    
inputs = {
            'the_input': X_data,
            'the_labels': Y_data,
            'input_length': input_length,
            'label_length': label_length,
            #'source_str': source_str
        }
#outputs = {'ctc': np.zeros([self.batch_size])}

XX = [X['the_input'], 
      X['the_labels'], 
      X['input_length'], 
      X['label_length']]
"""

#model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)

scores = loaded_model.evaluate()
for i in range(len(scores)):
    print("\n%s: %.2f%%" % (loaded_model.metrics_names[i], scores[i]*100))



ValueError: If evaluating from data tensors, you should specify the `steps` argument.

In [89]:
X=tiger_test.next_batch(K)
X['the_input']

TypeError: 'generator' object is not subscriptable